# Time Series Decomposition - Shell Sensor Data

This notebook performs MSTL (Multiple Seasonal-Trend decomposition using Loess) on Shell sensor data.

**Goal**: Extract hourly and daily seasonal patterns from sensor data.

**Steps**:
1. Load raw data
2. Select specific sensor(s)
3. Preprocess Value column (handle non-numeric values)
4. Check for missing timestamps and gaps
5. Interpolate gaps and report details
6. Filter to first 7 days
7. Apply MSTL decomposition with hourly + daily seasonality

# **Note**: It doesnt seem to be feasable to use 5-second intervall data as in this notebook because its too compute heavy even for one singular sensor. Also interpolating missing values with that intervall length would result in ~34% artificial values for the inspected sensor.

# Step 1: Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import json
from pathlib import Path

# Add SDK to path
SDK_PATH = Path().resolve().parents[2] / "src" / "sdk" / "python"
sys.path.insert(0, str(SDK_PATH))

# Import decomposition component
from rtdip_sdk.pipelines.decomposition.pandas import MSTLDecomposition

print("Imports complete")

## Step 2: Load Raw Data

**Important**: We load **raw data** (not preprocessed) because preprocessing removes outliers 

In [ ]:
# Path to raw data
data_path = Path().resolve().parent / "data" / "ShellData.parquet"

print(f"Loading: {data_path}")
print(f"File exists: {data_path.exists()}")
print()

# Load only the columns we need for memory efficiency
df = pd.read_parquet(data_path, columns=['TagName', 'EventTime', 'Value'])

print(f"Loaded {len(df):,} rows")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"Unique sensors: {df['TagName'].nunique():,}")
print(f"Columns: {df.columns.tolist()}")
print()
print("First few rows:")
df.head()

## Step 3: Select Specific Sensor


In [ ]:
# Select sensor
selected_sensor = "0SP116X5V:NXR.0"

print(f"Filtering to sensor: {selected_sensor}")
df_sensor = df[df['TagName'] == selected_sensor].copy()

print(f"Selected sensor has {len(df_sensor):,} rows")
print(f"Percentage of total data: {len(df_sensor)/len(df)*100:.2f}%")

# Clean up memory
del df

df_sensor.head()

## Step 4: Preprocess Value Column

The Value column may contain:
- Numeric values (what we want)
- Text strings like "Calc Failed" or "Bad"
- Special values like -1 (error indicators)

We need to:
1. Convert EventTime to datetime
2. Convert Value to numeric (coerce invalid values to NaN)
3. Remove error values and NaN

In [ ]:
print("Preprocessing Value column...")
print()

# Convert EventTime to datetime
print("1. Converting EventTime to datetime...")
df_sensor['EventTime'] = pd.to_datetime(df_sensor['EventTime'], errors='coerce')
invalid_times = df_sensor['EventTime'].isna().sum()
print(f"   Invalid timestamps: {invalid_times:,}")

# Drop invalid timestamps
if invalid_times > 0:
    df_sensor = df_sensor.dropna(subset=['EventTime']).copy()
    print(f"   Dropped {invalid_times:,} rows with invalid timestamps")
print()

# Convert Value to numeric
print("2. Converting Value to numeric...")
print(f"   Original dtype: {df_sensor['Value'].dtype}")

# Check if already numeric
if not pd.api.types.is_numeric_dtype(df_sensor['Value']):
    df_sensor['Value'] = pd.to_numeric(df_sensor['Value'], errors='coerce')
    print(f"   Converted to numeric")
else:
    print(f"   Already numeric")
print()

# Check for NaN after conversion
print("3. Checking for invalid values...")
nan_count = df_sensor['Value'].isna().sum()
error_count = (df_sensor['Value'] == -1).sum()
print(f"   NaN values: {nan_count:,} ({nan_count/len(df_sensor)*100:.2f}%)")
print(f"   Error values (-1): {error_count:,} ({error_count/len(df_sensor)*100:.2f}%)")
print()

# Remove NaN and error values
print("4. Removing invalid values...")
before_len = len(df_sensor)
df_sensor = df_sensor[
    (df_sensor['Value'].notna()) & 
    (df_sensor['Value'] != -1)
].copy()
removed = before_len - len(df_sensor)
print(f"   Removed {removed:,} rows ({removed/before_len*100:.2f}%)")
print(f"   Remaining: {len(df_sensor):,} rows")
print()

# Sort by time
print("5. Sorting by time...")
df_sensor = df_sensor.sort_values('EventTime').reset_index(drop=True)
print(f"  Sorted")
print()

# Show value statistics
print("Value statistics:")
print(df_sensor['Value'].describe())
print()
print(f"Time range: {df_sensor['EventTime'].min()} to {df_sensor['EventTime'].max()}")
print(f"Duration: {df_sensor['EventTime'].max() - df_sensor['EventTime'].min()}")

## Step 5: Analyze Sampling and Check for Missing Timestamps

We need to understand:
- What's the typical sampling interval?
- Where are the gaps in timestamps?
- How big are these gaps?

In [ ]:
print("Analyzing sampling intervals...")
print("="*80)

# Calculate time differences between consecutive points
time_diffs = df_sensor['EventTime'].diff().dropna()

# Statistics
print("\nSampling interval statistics:")
print(f"  Median:  {time_diffs.median()}")
print(f"  Mean:    {time_diffs.mean()}")
print(f"  Min:     {time_diffs.min()}")
print(f"  Max:     {time_diffs.max()}")
print()
print("Percentiles:")
for p in [25, 50, 75, 90, 95, 99]:
    print(f"  {p:2d}th: {time_diffs.quantile(p/100)}")

# Determine what's "normal" vs "gap"
median_interval = time_diffs.median()
print(f"\nMedian interval: {median_interval}")

# Define a gap as anything > 10x the median interval
gap_threshold = median_interval * 10
print(f"Gap threshold (10x median): {gap_threshold}")

# Find gaps
gaps = time_diffs[time_diffs > gap_threshold]
print(f"\n✓ Found {len(gaps)} gaps larger than {gap_threshold}")

if len(gaps) > 0:
    print("\nLargest gaps:")
    for i, (idx, gap_size) in enumerate(gaps.nlargest(10).items(), 1):
        gap_start = df_sensor.loc[idx-1, 'EventTime']
        gap_end = df_sensor.loc[idx, 'EventTime']
        print(f"  {i:2d}. {gap_size} from {gap_start} to {gap_end}")
else:
    print("\nNo significant gaps found!")

# Visualize sampling intervals
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

# Histogram of intervals
axes[0].hist(time_diffs.dt.total_seconds(), bins=100, edgecolor='black', alpha=0.7)
axes[0].axvline(median_interval.total_seconds(), color='red', linestyle='--', label=f'Median: {median_interval}')
axes[0].set_xlabel('Interval (seconds)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Sampling Intervals')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Intervals over time
axes[1].plot(df_sensor['EventTime'][1:], time_diffs.dt.total_seconds(), linewidth=0.5, alpha=0.7)
axes[1].axhline(median_interval.total_seconds(), color='red', linestyle='--', alpha=0.5, label='Median')
axes[1].axhline(gap_threshold.total_seconds(), color='orange', linestyle='--', alpha=0.5, label='Gap threshold')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Interval (seconds)')
axes[1].set_title('Sampling Intervals Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('sampling_analysis.png', dpi=150)
plt.show()

## Step 6: Resample to Regular Intervals

MSTL requires data at regular intervals. We'll:
1. Choose appropriate resampling frequency based on median interval
2. Create a regular time grid
3. Map our data to this grid

In [ ]:
print("Resampling to regular intervals...")
print("="*80)

# Determine appropriate resampling frequency
median_interval = df_sensor['EventTime'].diff().median()
print(f"Median interval: {median_interval}")

if median_interval <= pd.Timedelta(seconds=10):
    resample_freq = '5S'  # 5 seconds
    freq_name = '5 seconds'
elif median_interval <= pd.Timedelta(seconds=30):
    resample_freq = '30S'  # 30 seconds
    freq_name = '30 seconds'
elif median_interval <= pd.Timedelta(minutes=5):
    resample_freq = '1T'  # 1 minute
    freq_name = '1 minute'
else:
    resample_freq = '5T'  # 5 minutes
    freq_name = '5 minutes'

print(f"\nResampling frequency: {resample_freq} ({freq_name})")
print()

# Resample
print("Creating regular time grid...")
df_resampled = df_sensor.set_index('EventTime').resample(resample_freq)['Value'].mean().reset_index()

print(f"Resampled to {len(df_resampled):,} regular intervals")
print(f"Original points: {len(df_sensor):,}")
print(f"Regular intervals: {len(df_resampled):,}")

# Check how many are missing
missing_count = df_resampled['Value'].isna().sum()
print(f"\nMissing values after resampling: {missing_count:,} ({missing_count/len(df_resampled)*100:.2f}%)")

df_resampled.head(10)

## Step 7: Interpolate Gaps

We'll use **linear interpolation** to fill missing values, but with a limit:
- Small gaps (< 2 minutes): Interpolate
- Large gaps (≥ 2 minutes): Keep as NaN and handle later

This ensures we don't create unrealistic data across large time gaps.

In [ ]:
print("Interpolating gaps...")
print("="*80)

# Calculate interpolation limit based on resampling frequency
# We'll interpolate up to 2 minutes of missing data
max_gap_duration = pd.Timedelta(minutes=2)
resample_timedelta = pd.Timedelta(resample_freq)
interpolation_limit = int(max_gap_duration / resample_timedelta)

print(f"Resampling frequency: {resample_freq}")
print(f"Maximum gap to interpolate: {max_gap_duration} ({interpolation_limit} consecutive missing values)")
print()

# Before interpolation
missing_before = df_resampled['Value'].isna().sum()
print(f"Missing values before interpolation: {missing_before:,}")

# Find consecutive NaN sequences
is_nan = df_resampled['Value'].isna()
nan_groups = (is_nan != is_nan.shift()).cumsum()
nan_sequences = df_resampled[is_nan].groupby(nan_groups).size()

if len(nan_sequences) > 0:
    print(f"\nGap analysis before interpolation:")
    print(f"  Number of gaps: {len(nan_sequences)}")
    print(f"  Smallest gap: {nan_sequences.min()} points ({nan_sequences.min() * resample_timedelta})")
    print(f"  Largest gap: {nan_sequences.max()} points ({nan_sequences.max() * resample_timedelta})")
    print(f"  Mean gap size: {nan_sequences.mean():.1f} points ({nan_sequences.mean() * resample_timedelta})")
    
    # Show distribution of gap sizes
    print(f"\n  Gap size distribution:")
    small_gaps = (nan_sequences <= interpolation_limit).sum()
    large_gaps = (nan_sequences > interpolation_limit).sum()
    print(f"    ≤ {interpolation_limit} points (will interpolate): {small_gaps} gaps")
    print(f"    > {interpolation_limit} points (too large): {large_gaps} gaps")
    
    if large_gaps > 0:
        print(f"\n  Largest gaps (will NOT be interpolated):")
        for i, (group_id, size) in enumerate(nan_sequences.nlargest(5).items(), 1):
            gap_indices = df_resampled[is_nan & (nan_groups == group_id)].index
            gap_start = df_resampled.loc[gap_indices[0], 'EventTime']
            gap_end = df_resampled.loc[gap_indices[-1], 'EventTime']
            duration = size * resample_timedelta
            print(f"    {i}. {size} points ({duration}) from {gap_start} to {gap_end}")
print()

# Perform interpolation
print(f"Interpolating gaps up to {interpolation_limit} consecutive missing values...")
df_resampled['Value'] = df_resampled['Value'].interpolate(
    method='linear',
    limit=interpolation_limit,
    limit_direction='both'
)

# After interpolation
missing_after = df_resampled['Value'].isna().sum()
interpolated_count = missing_before - missing_after

print(f"\nInterpolation complete")
print(f"Interpolated: {interpolated_count:,} values ({interpolated_count/len(df_resampled)*100:.2f}%)")
print(f"Remaining NaN: {missing_after:,} values ({missing_after/len(df_resampled)*100:.2f}%)")

## Step 8: Handle Remaining Large Gaps

For any remaining NaN values (large gaps that weren't interpolated), we'll:
1. Identify continuous segments without NaN
2. Select the longest continuous segment
3. Use only that segment for decomposition

In [ ]:
print("Handling remaining gaps...")
print("="*80)

if df_resampled['Value'].isna().sum() > 0:
    print(f"Found {df_resampled['Value'].isna().sum():,} remaining NaN values")
    print("Finding longest continuous segment...\n")
    
    # Find continuous segments (groups of non-NaN values)
    is_valid = df_resampled['Value'].notna()
    segment_groups = (is_valid != is_valid.shift()).cumsum()
    
    # Get segments with valid data
    valid_segments = df_resampled[is_valid].groupby(segment_groups)
    segment_sizes = valid_segments.size()
    
    print(f"Found {len(segment_sizes)} continuous segments")
    print(f"\nTop 5 longest segments:")
    for i, (group_id, size) in enumerate(segment_sizes.nlargest(5).items(), 1):
        segment_data = df_resampled[is_valid & (segment_groups == group_id)]
        start_time = segment_data['EventTime'].min()
        end_time = segment_data['EventTime'].max()
        duration = end_time - start_time
        print(f"  {i}. {size:,} points ({duration}) from {start_time} to {end_time}")
    
    # Select longest segment
    longest_segment_id = segment_sizes.idxmax()
    df_continuous = df_resampled[is_valid & (segment_groups == longest_segment_id)].copy().reset_index(drop=True)
    
    print(f"\nSelected longest segment: {len(df_continuous):,} points")
    print(f"Start: {df_continuous['EventTime'].min()}")
    print(f"End: {df_continuous['EventTime'].max()}")
    print(f"Duration: {df_continuous['EventTime'].max() - df_continuous['EventTime'].min()}")
    
else:
    print("No remaining NaN values - using complete dataset")
    df_continuous = df_resampled.copy()

print(f"\nFinal dataset: {len(df_continuous):,} rows")
print(f"No missing values: {df_continuous['Value'].notna().all()}")

## Step 9: Filter to First 7 Days

Now we'll take the first 7 days of the continuous data for decomposition.

In [ ]:
print("Filtering to first 7 days...")
print("="*80)

# Get time range
start_time = df_continuous['EventTime'].min()
end_time = df_continuous['EventTime'].max()
total_duration = end_time - start_time

print(f"Full continuous segment:")
print(f"  Start: {start_time}")
print(f"  End: {end_time}")
print(f"  Duration: {total_duration}")
print(f"  Points: {len(df_continuous):,}")
print()

# Filter to first 7 days
seven_days_later = start_time + pd.Timedelta(days=7)
df_7days = df_continuous[df_continuous['EventTime'] <= seven_days_later].copy()

print(f"First 7 days:")
print(f"  Start: {df_7days['EventTime'].min()}")
print(f"  End: {df_7days['EventTime'].max()}")
print(f"  Duration: {df_7days['EventTime'].max() - df_7days['EventTime'].min()}")
print(f"  Points: {len(df_7days):,}")
print()

# Calculate periods
resample_timedelta = pd.Timedelta(resample_freq)
hourly_period = int(pd.Timedelta(hours=1) / resample_timedelta)
daily_period = int(pd.Timedelta(days=1) / resample_timedelta)

print(f"Periods for MSTL:")
print(f"  Hourly period: {hourly_period} observations per hour")
print(f"  Daily period: {daily_period} observations per day")
print()

print(f"Expected points for 7 days: {daily_period * 7:,}")
print(f"Actual points: {len(df_7days):,}")
print(f"Coverage: {len(df_7days) / (daily_period * 7) * 100:.1f}%")

# Check if we have enough data for decomposition
# MSTL requires at least 2 complete cycles of the largest period
min_required = daily_period * 2
if len(df_7days) >= min_required:
    print(f"\nSufficient data for decomposition (need >= {min_required:,} points)")
else:
    print(f"\nWARNING: Insufficient data (need >= {min_required:,} points, have {len(df_7days):,})")
    print(f"  Consider using more days or a coarser resampling frequency")

## Step 10: Visualize Data Before Decomposition

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.plot(df_7days['EventTime'], df_7days['Value'], linewidth=0.5, alpha=0.7)
ax.set_title(f'Sensor {selected_sensor} - First 7 Days (Ready for Decomposition)')
ax.set_xlabel('Time')
ax.set_ylabel('Value')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data_before_decomposition.png', dpi=150)
plt.show()

print(f"Data characteristics:")
print(f"  Mean: {df_7days['Value'].mean():.4f}")
print(f"  Std: {df_7days['Value'].std():.4f}")
print(f"  Min: {df_7days['Value'].min():.4f}")
print(f"  Max: {df_7days['Value'].max():.4f}")

## Step 11: Apply MSTL Decomposition with Hourly + Daily Seasonality

Now we'll decompose the time series into:
- **Trend**: Long-term direction
- **Seasonal (Hourly)**: Repeating pattern every hour
- **Seasonal (Daily)**: Repeating pattern every 24 hours
- **Residual**: Random noise

**Note**: MSTL can handle multiple seasonal periods simultaneously!

In [ ]:
print("Applying MSTL Decomposition...")
print("="*80)

print(f"Configuration:")
print(f"  Sensor: {selected_sensor}")
print(f"  Data points: {len(df_7days):,}")
print(f"  Resampling frequency: {resample_freq}")
print(f"  Hourly period: {hourly_period} observations")
print(f"  Daily period: {daily_period} observations")
print(f"  Duration: {df_7days['EventTime'].max() - df_7days['EventTime'].min()}")
print()

try:
    # Create MSTL decomposer with both hourly and daily seasonality
    print("Creating MSTL decomposer with hourly + daily seasonality...")
    mstl = MSTLDecomposition(
        df=df_7days,
        value_column='Value',
        timestamp_column='EventTime',
        periods=[hourly_period, daily_period],  # Both hourly and daily seasonality
        iterate=2
    )
    
    # Perform decomposition
    print("Decomposing (this may take a minute for 7 days of data)...")
    df_decomposed = mstl.decompose()
    
    print("\nDecomposition successful")
    print(f"\nComponents:")
    for col in df_decomposed.columns:
        if col not in ['EventTime', 'Value']:
            print(f"  - {col}")
    
    print(f"\nShape: {df_decomposed.shape}")
    
except Exception as e:
    print(f"\nDecomposition failed")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
    df_decomposed = None

## Step 12: Visualize Decomposition Results

In [ ]:
if df_decomposed is not None:
    hourly_col = f'seasonal_{hourly_period}'
    daily_col = f'seasonal_{daily_period}'
    
    fig, axes = plt.subplots(5, 1, figsize=(16, 15))
    
    # Original
    axes[0].plot(df_decomposed['EventTime'], df_decomposed['Value'], linewidth=0.5, alpha=0.7)
    axes[0].set_title(f'Original - {selected_sensor} (First 7 Days)')
    axes[0].set_ylabel('Value')
    axes[0].grid(True, alpha=0.3)
    
    # Trend
    axes[1].plot(df_decomposed['EventTime'], df_decomposed['trend'], linewidth=1.5, color='orange')
    axes[1].set_title('Trend Component')
    axes[1].set_ylabel('Trend')
    axes[1].grid(True, alpha=0.3)
    
    # Seasonal (Hourly)
    axes[2].plot(df_decomposed['EventTime'], df_decomposed[hourly_col], linewidth=0.5, color='blue')
    axes[2].set_title(f'Hourly Seasonal Component (Period = {hourly_period} points = 1 hour)')
    axes[2].set_ylabel('Hourly Seasonal')
    axes[2].grid(True, alpha=0.3)
    
    # Seasonal (Daily)
    axes[3].plot(df_decomposed['EventTime'], df_decomposed[daily_col], linewidth=0.5, color='green')
    axes[3].set_title(f'Daily Seasonal Component (Period = {daily_period} points = 24 hours)')
    axes[3].set_ylabel('Daily Seasonal')
    axes[3].grid(True, alpha=0.3)
    
    # Residual
    axes[4].plot(df_decomposed['EventTime'], df_decomposed['residual'], linewidth=0.5, color='red', alpha=0.5)
    axes[4].set_title('Residual Component')
    axes[4].set_xlabel('Time')
    axes[4].set_ylabel('Residual')
    axes[4].grid(True, alpha=0.3)
    
    plt.tight_layout()
    safe_tag = selected_sensor.replace('/', '_').replace('\\', '_').replace(':', '_')
    plt.savefig(f'decomposition_{safe_tag}.png', dpi=150)
    plt.show()
else:
    print("Cannot visualize - decomposition failed")

## Step 13: Variance Analysis

How much of the total variance is explained by each component?

In [ ]:
if df_decomposed is not None:
    print("Variance Analysis")
    print("="*80)
    
    hourly_col = f'seasonal_{hourly_period}'
    daily_col = f'seasonal_{daily_period}'
    
    total_var = df_decomposed['Value'].var()
    trend_var = df_decomposed['trend'].dropna().var()
    hourly_var = df_decomposed[hourly_col].dropna().var()
    daily_var = df_decomposed[daily_col].dropna().var()
    residual_var = df_decomposed['residual'].dropna().var()
    
    print(f"\nTotal variance: {total_var:.6f}")
    print()
    print(f"Component breakdown:")
    print(f"  Trend:           {trend_var:>12.6f} ({trend_var/total_var*100:>6.2f}%)")
    print(f"  Hourly Seasonal: {hourly_var:>12.6f} ({hourly_var/total_var*100:>6.2f}%)")
    print(f"  Daily Seasonal:  {daily_var:>12.6f} ({daily_var/total_var*100:>6.2f}%)")
    print(f"  Residual:        {residual_var:>12.6f} ({residual_var/total_var*100:>6.2f}%)")
    print()
    
    combined_seasonal = hourly_var + daily_var
    print(f"  Combined Seasonal: {combined_seasonal:>12.6f} ({combined_seasonal/total_var*100:>6.2f}%)")
else:
    print("Cannot analyze variance - decomposition failed")

## Step 14: Zoom into First 24 Hours

In [ ]:
if df_decomposed is not None:
    # Take first 24 hours
    start = df_decomposed['EventTime'].min()
    end = start + pd.Timedelta(days=1)
    
    df_day = df_decomposed[df_decomposed['EventTime'] <= end].copy()
    hourly_col = f'seasonal_{hourly_period}'
    daily_col = f'seasonal_{daily_period}'
    
    fig, axes = plt.subplots(3, 1, figsize=(16, 10))
    
    # Original vs Trend
    axes[0].plot(df_day['EventTime'], df_day['Value'], linewidth=0.5, alpha=0.7, label='Original')
    axes[0].plot(df_day['EventTime'], df_day['trend'], linewidth=2, color='orange', label='Trend')
    axes[0].set_title('Original vs Trend (First 24 Hours)')
    axes[0].set_ylabel('Value')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Hourly seasonal pattern
    axes[1].plot(df_day['EventTime'], df_day[hourly_col], linewidth=1, color='blue')
    axes[1].set_title('Hourly Seasonal Pattern (First 24 Hours)')
    axes[1].set_ylabel('Hourly Seasonal')
    axes[1].grid(True, alpha=0.3)
    
    # Daily seasonal pattern
    axes[2].plot(df_day['EventTime'], df_day[daily_col], linewidth=1.5, color='green')
    axes[2].set_title('Daily Seasonal Pattern (First 24 Hours)')
    axes[2].set_xlabel('Time')
    axes[2].set_ylabel('Daily Seasonal')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('daily_pattern_zoom.png', dpi=150)
    plt.show()
else:
    print("Cannot visualize patterns - decomposition failed")

## Step 15: Save Results

In [ ]:
if df_decomposed is not None:
    print("Saving results...")
    print("="*80)
    
    safe_tag = selected_sensor.replace('/', '_').replace('\\', '_').replace(':', '_')
    hourly_col = f'seasonal_{hourly_period}'
    daily_col = f'seasonal_{daily_period}'
    
    # Save decomposed data
    parquet_file = f'decomposition_{safe_tag}_7days.parquet'
    df_decomposed.to_parquet(parquet_file, index=False)
    print(f"Saved: {parquet_file}")
    
    # Save metadata
    total_var = df_decomposed['Value'].var()
    trend_var = df_decomposed['trend'].dropna().var()
    hourly_var = df_decomposed[hourly_col].dropna().var()
    daily_var = df_decomposed[daily_col].dropna().var()
    residual_var = df_decomposed['residual'].dropna().var()
    
    metadata = {
        'sensor': selected_sensor,
        'duration': '7 days',
        'data_points': len(df_decomposed),
        'resampling_frequency': resample_freq,
        'periods': {
            'hourly': hourly_period,
            'daily': daily_period
        },
        'time_range': {
            'start': str(df_decomposed['EventTime'].min()),
            'end': str(df_decomposed['EventTime'].max())
        },
        'variance_explained': {
            'trend_pct': float(trend_var / total_var * 100),
            'hourly_seasonal_pct': float(hourly_var / total_var * 100),
            'daily_seasonal_pct': float(daily_var / total_var * 100),
            'combined_seasonal_pct': float((hourly_var + daily_var) / total_var * 100),
            'residual_pct': float(residual_var / total_var * 100)
        },
        'preprocessing': {
            'source': 'raw data (not preprocessed)',
            'interpolation_limit': f'{max_gap_duration} ({interpolation_limit} points)',
            'gaps_interpolated': int(interpolated_count)
        }
    }
    
    metadata_file = f'decomposition_{safe_tag}_7days.json'
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved: {metadata_file}")
    
    print("\nComplete")
    print(f"\nSummary:")
    print(f"  Sensor: {selected_sensor}")
    print(f"  Duration: 7 days")
    print(f"  Hourly seasonality: {hourly_var/total_var*100:.2f}% of variance")
    print(f"  Daily seasonality: {daily_var/total_var*100:.2f}% of variance")
    print(f"  Combined seasonal: {(hourly_var + daily_var)/total_var*100:.2f}% of variance")
else:
    print("Cannot save results - decomposition failed")